<a href="https://colab.research.google.com/github/ThuyHaLE/ThuyHaLE/blob/main/Dash_Plotly_Covid19_info_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TOOLs

In [ ]:
!pip install --upgrade plotly
!pip install jupyter-dash
!pip install dash-cytoscape

In [4]:
import pandas as pd
import numpy as np
import math
import datetime

In [5]:
import plotly.graph_objects as go
import plotly.express as px

In [6]:
from jupyter_dash import JupyterDash

import dash
from dash import no_update, dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output 
import dash_cytoscape as cyto

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  


#DATA

##UPDATED DATA BY WHO (https://covid19.who.int)

In [7]:
#Open the file and read it into a variable

#1. Daily cases and deaths by date reported to WHO
daily_covid_19 = pd.read_csv(
    'https://covid19.who.int/WHO-COVID-19-global-data.csv')
daily_covid_19 = daily_covid_19.rename(columns = {'Country_code':'Alpha-2 code'})

#2. Latest reported counts of cases and deaths
count_covid_19 = pd.read_csv(
    'https://covid19.who.int/WHO-COVID-19-global-table-data.csv')
count_covid_19.index.names = ['Country']
count_covid_19.reset_index (inplace=True)
count_covid_19 = count_covid_19.rename(columns = {'Name':'Subregion'})
count_covid_19['Subregion'] = count_covid_19['Subregion'].fillna('Global')

In [8]:
#Take a look at the data (Latest reported counts of cases and deaths)
count_covid_19.head()

,Country,Subregion,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,Deaths - newly reported in last 7 days,Deaths - newly reported in last 7 days per 100000 population,Deaths - newly reported in last 24 hours
0,Global,Global,620878405,7965.555073,3034234,38.927683,454440,6543138,83.945142,8473,0.108704,1361,NaN
1,United States of America,Americas,95529652,28860.691000,267056,80.681000,57517,1052823,318.071000,2366,0.715000,709,NaN
2,India,South-East Asia,44623997,3233.613000,17537,1.271000,2678,528857,38.323000,103,0.007000,10,NaN
3,France,Europe,35060107,53905.985000,396331,609.371000,64198,152157,233.946000,368,0.566000,62,NaN
4,Brazil,Americas,34736653,16342.091000,37636,17.706000,5114,687026,323.216000,453,0.213000,63,NaN


In [9]:
#Take a look at the data (Daily cases and deaths by date reported to WHO)
daily_covid_19.head()

,Date_reported,Alpha-2 code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


#VISUALIZATION

###FOR THE "Daily cases and deaths by date reported to WHO"

In [10]:
#GET A COPY OF DATA
data = daily_covid_19.copy()
data['Date_reported'] = pd.to_datetime(data['Date_reported'])
data.head()

,Date_reported,Alpha-2 code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


####Daily cases and deaths by date reported to WHO (table)


In [24]:
#Daily cases and deaths by date reported to WHO
app = JupyterDash(__name__)

colors = {'background': '#111111','text': '#7FDBFF'}
tabs_styles = {'height': '30px'}
tab_style = {'borderBottom': '1px solid #d6d6d6',
             'padding': '6px'}
tab_selected_style = {'borderTop': '1px solid #d6d6d6',
                      'borderBottom': '1px solid #d6d6d6',
                      'backgroundColor': '#119DFF',
                      'fontWeight': 'bold',
                      'color': 'blue','padding': '6px'}

app.layout = html.Div([
    html.H4('Daily cases and deaths by date reported to WHO',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
    dcc.Store(id='intermediate-value'),
    html.Br(),
    dcc.Tabs(id='tabs-1', value='Country', children=[
        dcc.Tab(label='Daily cases and deaths by country', value='Country', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Daily cases and deaths by date', value='Date_reported', style=tab_style, selected_style=tab_selected_style)]),
        html.Div(id='tabs-content-1',style={'color': 'Gold', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    dcc.Dropdown(id="dropdown",options=[]),
    html.Div(id='dropdown-content-1',style={'color': 'Gold', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    dash_table.DataTable(id='table-filtering-be',
                         columns=[{'name': 'Country', 'id': 'Country', 'type': 'text'},
                                  {'name': 'Alpha-2 code', 'id': 'Alpha-2 code', 'type': 'text'},
                                  {'name': 'WHO_region', 'id': 'WHO_region', 'type': 'text'},
                                  {'name': 'Date_reported', 'id': 'Date_reported', 'type': 'datetime'},
                                  {'name': 'New_cases', 'id': 'New_cases', 'type': 'numeric'},
                                  {'name': 'Cumulative_cases', 'id': 'Cumulative_cases', 'type': 'numeric'},
                                  {'name': 'New_deaths', 'id': 'New_deaths', 'type': 'numeric'},
                                  {'name': 'Cumulative_deaths', 'id': 'Cumulative_deaths', 'type': 'numeric'}],
                         data=[],page_size=10, editable=True,page_action="native",
                         #filter_action='custom',filter_query='',
                         sort_action="native",sort_mode="multi",
                         style_as_list_view=True,fixed_rows={'headers': True},style_table={'height': '350px', 'overflowY': 'auto'},
                         style_cell={'height': 'auto','minWidth': '40px', 'width': '60px', 'maxWidth': '80px','whiteSpace': 'normal'},
                         style_header={'font_family': 'Rockwell','font_size': '10px','color': 'black','backgroundColor': 'white',
                                       'fontWeight': 'bold','border': '1px solid black','whiteSpace': 'normal','height': 'auto',
                                       'text_align': 'center','padding': '5px'},
                         style_data={'font_family': 'Rockwell','font_size': '8px','color': 'blue','backgroundColor': 'white',
                                     'whiteSpace': 'normal','height': 'auto'},),
    ])

@app.callback(
    Output('tabs-content-1', 'children'),
    Input('tabs-1', 'value'))
def update_output(value):
  return 'You have selected Daily cases and deaths by {}'.format(value)

@app.callback(
    Output('dropdown-content-1', 'children'),
    Input('dropdown', 'value'))
def update_Drop(value):
  return 'You have selected Daily cases and deaths by {}'.format(value)

@app.callback(
    Output("dropdown", "options"),
    Input('tabs-1', "value"))
def updateDropdown(value):
  if value=='Country':
    options = [{'label': i, 'value': i} for i in data[value].unique()]
    return options
  elif value=='Date_reported':
    options = [{'label': np.datetime_as_string(i, unit='D'), 
                'value': np.datetime_as_string(i, unit='D')} for i in data['Date_reported'].unique()]
    return options
  
@app.callback(
    Output("table-filtering-be", "data"),
    Input('tabs-1', "value"),
    Input("dropdown", "value"))
def updateTable(tabs, dropdown):
  if tabs=='Country':
    df = data[data[tabs]==dropdown]
    return df.to_dict('records')
  elif tabs=='Date_reported':
    df = data[data[tabs]==dropdown]
    return df.to_dict('records')

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

####Daily cases and deaths by date reported to WHO (bar & scatter)



In [25]:
colors = {'background': '#111111','text': '#7FDBFF'}
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H4('Daily cases and deaths by date reported to WHO',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
    html.I("Choose a DatePickerRange to observe daily cases and deaths by date.",
           style={'textAlign': 'left', 'color': 'white', 'font-size': 15}),
    html.Hr(),
    dcc.DatePickerRange(id='my-date-picker-range',
                        minimum_nights=5,
                        min_date_allowed=data['Date_reported'].min(),
                        max_date_allowed=data['Date_reported'].max(),
                        display_format='Y-M-D',
                        start_date=data['Date_reported'].max().strftime("%Y-%m-%d"),
                        end_date=data['Date_reported'].max().strftime("%Y-%m-%d")),
    html.Hr(),
    html.Div(id='output-container-date-picker-range',style={'color': 'white', 'font-size': 12}),
    html.Br(),
    dcc.Graph(id="graph"),
    ])

@app.callback(
    Output("output-container-date-picker-range", "children"),
    Input("my-date-picker-range", "start_date"),
    Input("my-date-picker-range", "end_date"),)
def number_render(start_date, end_date):
    return "From: {} To: {}.".format(start_date, end_date)

@app.callback(Output("graph", "figure"),
              Input("my-date-picker-range", "start_date"),
              Input("my-date-picker-range", "end_date"),)
def update_graph(start_date,end_date):
  df_New_cases = data[(data['Date_reported']>=start_date) & (data['Date_reported']<=end_date)].groupby('Country')['New_cases'].sum().reset_index()
  df_New_deaths = data[(data['Date_reported']>=start_date) & (data['Date_reported']<=end_date)].groupby('Country')['New_deaths'].sum().reset_index()
  from plotly.subplots import make_subplots
  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(go.Bar(x=df_New_cases["Country"], y=df_New_cases['New_cases']),secondary_y=False)
  fig.update_traces(name='New_cases by country', showlegend = True,secondary_y=False)
  fig.add_trace(go.Scatter(x=df_New_deaths["Country"], y=df_New_deaths['New_deaths'],mode='lines+markers'),secondary_y=True)
  fig.update_traces(name='New_deaths by country', showlegend = True,secondary_y=True)
  fig.update_layout(title_text="Daily cases and deaths by date reported to WHO from {} to {}".format(start_date,end_date),
                    titlefont = dict(family = 'Rockwell',size = 15,color = 'Gold'))
  fig.update_xaxes(tickangle=45, title_text="Country",color='black')
  fig.update_yaxes(title_text="New_cases", titlefont = dict(family = 'Rockwell',size = 10,color = 'blue'), secondary_y=False)
  fig.update_yaxes(title_text="New_deaths", titlefont = dict(family = 'Rockwell',size = 10,color = 'red'), secondary_y=True)
  fig.add_annotation(x=-13, y=round(df_New_cases['New_cases'].mean(),2),
                     text="New cases-average: {}".format(round(df_New_cases['New_cases'].mean(),2)), 
                     showarrow=False,font=dict(color="blue",size=8),)
  fig.add_annotation(x=-10, y=df_New_cases['New_cases'].max(),
                     text="New cases-max: {}, New cases-min: {}".format(df_New_cases['New_cases'].max(),df_New_cases['New_cases'].min()), 
                     showarrow=False,font=dict(color="blue",size=8),)
  fig.add_annotation(x=248, y=round(df_New_deaths['New_deaths'].mean(),2),
                     text="New deaths-average: {}".format(round(df_New_deaths['New_deaths'].mean(),2)),
                     xref="x", yref="y2", showarrow=False,font=dict(color="red",size=8),)
  fig.add_annotation(x=245, y=df_New_deaths['New_deaths'].max(),
                     text="New deaths-max: {}, New deaths-min: {}".format(df_New_deaths['New_deaths'].max(),df_New_deaths['New_deaths'].min()),
                     xref="x", yref="y2", showarrow=False,font=dict(color="red",size=8),)
  fig.update_layout(font=dict(family='Rockwell', size=6, color='crimson'),width = 2500,height=500)
  return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

####Daily cases and deaths by date reported to WHO (pie)

In [26]:
data_3 = data.groupby(['Country','Date_reported'])[['New_cases','New_deaths']].sum().reset_index()
data_3['Date_reported (year)'] = data_3['Date_reported'].dt.strftime('%Y')
data_3['Date_reported (month)'] = data_3['Date_reported'].dt.strftime('%m')
sunburst_data = data_3[(data_3['Date_reported (year)']=='2021') & (data_3['Date_reported (month)']=='01')].groupby(
    ['Country'])[['New_cases','New_deaths']].sum().reset_index()

fig = px.pie(sunburst_data[sunburst_data['New_cases']>0].sort_values(by = ['New_cases'], ascending = True).head(5), 
             values='New_cases' , names='Country' , title='New_cases by Country', hole=.3, 
             #hover_data=['New_cases','Date_reported (month)'], 
             #labels=['Date_reported (month)']
             )
fig.show()

In [27]:
#Pie Chart
data_2 = data.groupby(['Country','Date_reported'])[['New_cases','New_deaths']].sum().reset_index()
data_2['Date_reported (year)'] = data_2['Date_reported'].dt.strftime('%Y')
data_2['Date_reported (month)'] = data_2['Date_reported'].dt.strftime('%m')
pie_data = data_2[(data_2['Country']=='Viet Nam') & (data_2['Date_reported (year)']=='2021')].groupby(
    ['Date_reported (month)'])['New_cases'].sum().reset_index()

fig = px.pie(pie_data, values='New_cases' , names='Date_reported (month)' , title='New_cases by month', hole=.3, 
             #hover_data=['New_cases','Date_reported (month)'], 
             labels=['Date_reported (month)'])
fig.show()

In [28]:
#Daily cases by date reported to WHO
app = JupyterDash(__name__)

colors = {'background': '#FFFFFF','text': '#111111'}
tabs_styles = {'height': '30px'}
tab_style = {'borderBottom': '1px solid #d6d6d6',
             'padding': '6px'}
tab_selected_style = {'borderTop': '1px solid #d6d6d6',
                      'borderBottom': '1px solid #d6d6d6',
                      'backgroundColor': '#119DFF',
                      'fontWeight': 'bold',
                      'color': 'blue','padding': '6px'}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H4('Daily cases by date reported to WHO',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
    html.Br(),
    dcc.Tabs(id='tabs-1', value='Country', children=[
        dcc.Tab(label='Daily cases by country and year', value='Country', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Top 10 countries with daily cases by year-month', value='Date_reported', style=tab_style, selected_style=tab_selected_style)]),
        html.Div(id='tabs-content-1',style={'color': 'Gold', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    html.Div(["Dropdown1. Please choose one option to show the result!",
              dcc.Dropdown(id="my-dynamic-dropdown",options=[],),]),
    html.Br(),
    html.Div(["Dropdown2. Please choose one option to show the result!",
              dcc.Dropdown(id="my-multi-dynamic-dropdown",options=[],),]),
    html.Br(),
    dcc.Graph(id="graph"),
    ])

@app.callback(
    Output('my-dynamic-dropdown', 'options'),
    Input('tabs-1', 'value'))
def update_Drop(value):
    if value=='Country':
      options=[{'label': i, 'value': i} for i in data['Country'].unique()]
      return options
    elif value=='Date_reported':
      options=[{'label': i, 'value': i} for i in data['Date_reported'].dt.strftime('%Y').unique()]
      return options

@app.callback(
    Output('my-multi-dynamic-dropdown', 'options'),
    Input('tabs-1', 'value'))
def update_Drop(value):
    if value=='Country':
      options=[{'label': i, 'value': i} for i in data['Date_reported'].dt.strftime('%Y').unique()]
      return options
    elif value=='Date_reported':
      options=[{'label': i, 'value': i} for i in data['Date_reported'].dt.strftime('%m').unique()]
      return options

@app.callback(
    Output("graph", "figure"),
    Input('tabs-1', 'value'),
    Input('my-dynamic-dropdown', 'value'),
    Input('my-multi-dynamic-dropdown', 'value'),)
def clean_data(tab, drop1, drop2):
  import datetime as dt
  df = data.groupby(['Country','Date_reported'])[['New_cases','New_deaths']].sum().reset_index()
  df['Date_reported (year)'] = df['Date_reported'].dt.strftime('%Y')
  df['Date_reported (month)'] = df['Date_reported'].dt.strftime('%m')
  if tab=='Country':
      pie_data = df[(df['Country']==drop1) & (df['Date_reported (year)']==drop2)].groupby(['Date_reported (month)'])['New_cases'].sum().reset_index()
      fig = px.pie(pie_data, values='New_cases' , names='Date_reported (month)' , 
                   title='Daily cases of {} in {} by month'.format(drop1,drop2),
                   hole=.3, 
                   labels=['Date_reported (month)'])
      fig.update_layout(font=dict(family='Rockwell', size=10, color='crimson'),)
      return fig
  elif tab=='Date_reported':
      pie_data = df[(df['Date_reported (year)']==drop1) & (df['Date_reported (month)']==drop2)].groupby(['Country'])[['New_cases','New_deaths']].sum().reset_index()
      fig = px.pie(pie_data[pie_data['New_cases']>0].sort_values(by = ['New_cases'], ascending = False).head(10), 
             values='New_cases' , names='Country' , 
             title='Top 10 countries with daily cases in {}/{}'.format(drop1,drop2), 
             hole=.3, )
      fig.update_layout(font=dict(family='Rockwell', size=10, color='crimson'),)
      return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

In [29]:
#Daily deaths by date reported to WHO
app = JupyterDash(__name__)

colors = {'background': '#FFFFFF','text': '#111111'}
tabs_styles = {'height': '30px'}
tab_style = {'borderBottom': '1px solid #d6d6d6',
             'padding': '6px'}
tab_selected_style = {'borderTop': '1px solid #d6d6d6',
                      'borderBottom': '1px solid #d6d6d6',
                      'backgroundColor': '#119DFF',
                      'fontWeight': 'bold',
                      'color': 'blue','padding': '6px'}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H4('Daily deaths by date reported to WHO',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
    html.Br(),
    dcc.Tabs(id='tabs-1', value='Country', children=[
        dcc.Tab(label='Daily deaths by country and year', value='Country', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Top 10 countries with daily deaths by year-month', value='Date_reported', style=tab_style, selected_style=tab_selected_style)]),
        html.Div(id='tabs-content-1',style={'color': 'Gold', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    html.Div(["Dropdown1. Please choose one option to show the result!",
              dcc.Dropdown(id="my-dynamic-dropdown",options=[],),]),
    html.Br(),
    html.Div(["Dropdown2. Please choose one option to show the result!",
              dcc.Dropdown(id="my-multi-dynamic-dropdown",options=[],),]),
    html.Br(),
    dcc.Graph(id="graph"),
    ])

@app.callback(
    Output('my-dynamic-dropdown', 'options'),
    Input('tabs-1', 'value'))
def update_Drop(value):
    if value=='Country':
      options=[{'label': i, 'value': i} for i in data['Country'].unique()]
      return options
    elif value=='Date_reported':
      options=[{'label': i, 'value': i} for i in data['Date_reported'].dt.strftime('%Y').unique()]
      return options

@app.callback(
    Output('my-multi-dynamic-dropdown', 'options'),
    Input('tabs-1', 'value'))
def update_Drop(value):
    if value=='Country':
      options=[{'label': i, 'value': i} for i in data['Date_reported'].dt.strftime('%Y').unique()]
      return options
    elif value=='Date_reported':
      options=[{'label': i, 'value': i} for i in data['Date_reported'].dt.strftime('%m').unique()]
      return options

@app.callback(
    Output("graph", "figure"),
    Input('tabs-1', 'value'),
    Input('my-dynamic-dropdown', 'value'),
    Input('my-multi-dynamic-dropdown', 'value'),)
def clean_data(tab, drop1, drop2):
  import datetime as dt
  df = data.groupby(['Country','Date_reported'])[['New_cases','New_deaths']].sum().reset_index()
  df['Date_reported (year)'] = df['Date_reported'].dt.strftime('%Y')
  df['Date_reported (month)'] = df['Date_reported'].dt.strftime('%m')
  if tab=='Country':
      pie_data = df[(df['Country']==drop1) & (df['Date_reported (year)']==drop2)].groupby(['Date_reported (month)'])['New_deaths'].sum().reset_index()
      fig = px.pie(pie_data, values='New_deaths' , names='Date_reported (month)' , 
                   title='Daily deaths of {} in {} by month'.format(drop1,drop2),
                   hole=.3, 
                   labels=['Date_reported (month)'])
      fig.update_layout(font=dict(family='Rockwell', size=10, color='crimson'),)
      return fig
  elif tab=='Date_reported':
      pie_data = df[(df['Date_reported (year)']==drop1) & (df['Date_reported (month)']==drop2)].groupby(['Country'])[['New_cases','New_deaths']].sum().reset_index()
      fig = px.pie(pie_data[pie_data['New_deaths']>0].sort_values(by = ['New_deaths'], ascending = False).head(10), 
             values='New_deaths' , names='Country' , 
             title='Top 10 countries with daily deaths in {}/{}'.format(drop1,drop2), 
             hole=.3, )
      fig.update_layout(font=dict(family='Rockwell', size=10, color='crimson'),)
      return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

####Daily cases and deaths by date reported to WHO (line)

In [30]:
#Bar Plot
data_1 = data.groupby(['Country','Date_reported'])[['New_cases','New_deaths']].sum().reset_index()
data_1['Date_reported (year)'] = data_1['Date_reported'].dt.strftime('%Y')
line_data = data_1[data_1['Country']=='Viet Nam'].groupby(['Date_reported (year)'])['New_cases'].sum().reset_index()

fig = go.Figure(data=go.Bar(x=line_data['Date_reported (year)'], y=line_data['New_cases'],
                            text=line_data['New_cases'],textposition="outside"))
fig.update_layout(title='New_cases by year', xaxis_title='Date_reported (year)', yaxis_title='New_cases',
                  font=dict(family='Rockwell', size=10, color='crimson'))
fig.update_xaxes(title_text="Date_reported (year)",color='black')
fig.show()

In [31]:
#Line Plot
data_1 = data.groupby(['Country','Date_reported'])[['New_cases','New_deaths']].sum().reset_index()
line_data = data_1[data_1['Country']=='Viet Nam'].groupby(['Date_reported'])['New_cases'].sum().reset_index()

fig = go.Figure(data=go.Scatter(x=line_data['Date_reported'], y=line_data['New_cases'],
                                mode='lines', marker=dict(color='blue')))
fig.update_layout(title='New_cases by date', xaxis_title='Date_reported', yaxis_title='New_cases',
                  hovermode='x unified',
                  font=dict(family='Rockwell', size=10, color='crimson'))
fig.update_xaxes(tickangle=45, title_text="Date_reported",color='black')
fig.show()

In [32]:
#Scatter Plot
data_1 = data.groupby(['Country','Date_reported'])[['New_cases','New_deaths']].sum().reset_index()
data_1['Date_reported (year-month)'] = data_1['Date_reported'].dt.strftime('%Y-%m')
line_data = data_1[data_1['Country']=='Viet Nam'].groupby(['Date_reported (year-month)'])['New_cases'].sum().reset_index()

fig = go.Figure(data=go.Scatter(x=line_data['Date_reported (year-month)'], y=line_data['New_cases'],
                                hoverinfo='none', 
                                mode='lines+markers', marker=dict(color='green')))
fig.update_layout(title='New_cases by year-month', xaxis_title='Date_reported (year-month)', yaxis_title='New_cases',
                  font=dict(family='Rockwell', size=10, color='crimson'))
fig.update_xaxes(tickangle=45, title_text="Date_reported (year-month)",color='black')
fig.update_traces(hovertemplate="%{y}",
                  name='New cases', showlegend = True)
fig.show()

In [33]:
#Daily cases by date reported to WHO
app = JupyterDash(__name__)

colors = {'background': '#FFFFFF','text': '#111111'}
tabs_styles = {'height': '30px'}
tab_style = {'borderBottom': '1px solid #d6d6d6',
             'padding': '6px'}
tab_selected_style = {'borderTop': '1px solid #d6d6d6',
                      'borderBottom': '1px solid #d6d6d6',
                      'backgroundColor': '#119DFF',
                      'fontWeight': 'bold',
                      'color': 'blue','padding': '6px'}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H4('Daily cases by date reported to WHO',style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
    html.Br(),
    html.Div(["Please choose the country to show the result!",
              dcc.Dropdown(id="my-dynamic-dropdown",value = 'Viet Nam',
                           options=[{'label': i, 'value': i} for i in data['Country'].unique()],),]),
    dcc.Tabs(id='tabs-1', value='New_cases', children=[
        dcc.Tab(label='New cases', value='New_cases', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='New deaths', value='New_deaths', style=tab_style, selected_style=tab_selected_style)]),
        html.Div(id='tabs-content-1',style={'color': 'Gold', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    dcc.RadioItems(id='mode',inline=True,options=['by year','by year-month', 'by date',],value='by date'),
    dcc.Graph(id="graph"),
    ])

@app.callback(
    Output("graph", "figure"),
    Input('my-dynamic-dropdown', 'value'),
    Input('tabs-1', 'value'),
    Input('mode', 'value'))
def clean_data(dropdown,tab,check):
    import datetime as dt
    if check == 'by year-month':
        df = data.groupby(['Country','Date_reported'])[['New_cases','New_deaths']].sum().reset_index()
        df['Date_reported (year-month)'] = df['Date_reported'].dt.strftime('%Y-%m')
        line_data = df[df['Country']==dropdown].groupby(['Date_reported (year-month)'])[tab].sum().reset_index()
        fig = go.Figure(data=go.Scatter(x=line_data['Date_reported (year-month)'], y=line_data[tab],
                                    hoverinfo='none', 
                                    mode='lines+markers', marker=dict(color='green')))
        fig.update_traces(name='{}'.format(tab), showlegend = True,
                          hovertemplate="%{y}")
        fig.update_layout(title='{} in {} by year-month'.format(tab,dropdown), 
                          xaxis_title='Date_reported (year-month)', yaxis_title=tab,
                          font=dict(family='Rockwell', size=10, color='crimson'))
        fig.add_hline(y=line_data[tab].mean(), annotation_text="{} - average: {}".format(tab,round(line_data[tab].mean(),2)), 
                      annotation_position="top left",
                      line_width=1, line_dash='dash',line_color='Red',yref='paper')
        return fig
    elif check == 'by date':
        df = data.groupby(['Country','Date_reported'])[['New_cases','New_deaths']].sum().reset_index()
        line_data = df[df['Country']==dropdown].groupby(['Date_reported'])[tab].sum().reset_index()
        fig = go.Figure(data=go.Scatter(x=line_data['Date_reported'], y=line_data[tab],
                                        mode='lines', marker=dict(color='blue')))
        fig.update_layout(title='{} in {} by date'.format(tab,dropdown), 
                          xaxis_title='Date_reported', yaxis_title=tab,
                          hovermode='x unified',
                          font=dict(family='Rockwell', size=10, color='crimson'))
        fig.add_hline(y=line_data[tab].mean(), annotation_text="{} - average: {}".format(tab,round(line_data[tab].mean(),2)), 
                      annotation_position="top left",
                      line_width=1, line_dash='dash',line_color='Red',yref='paper')
        fig.update_xaxes(tickangle=45, title_text="Date_reported",color='black')
        return fig
    elif check== 'by year':
        df = data.groupby(['Country','Date_reported'])[['New_cases','New_deaths']].sum().reset_index()
        df['Date_reported (year)'] = df['Date_reported'].dt.strftime('%Y')
        line_data = df[df['Country']==dropdown].groupby(['Date_reported (year)'])[tab].sum().reset_index()
        fig = go.Figure(data=go.Bar(x=line_data['Date_reported (year)'], y=line_data[tab],
                                    text=line_data[tab],textposition="outside"))
        fig.update_layout(title='{} in {} by year'.format(tab,dropdown), 
                          xaxis_title='Date_reported (year)', yaxis_title=tab,
                          font=dict(family='Rockwell', size=10, color='crimson'))
        fig.update_xaxes(title_text="Date_reported (year)",color='black')
        fig.add_hline(y=line_data[tab].mean(), annotation_text="{} - average: {}".format(tab,round(line_data[tab].mean(),2)), 
                      annotation_position="top left",
                      line_width=1, line_dash='dash',line_color='Red',yref='paper')
        return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

####Daily cases and deaths by date reported to WHO (bar)

In [34]:
#Daily cases and deaths by date reported to WHO
colors = {'background': '#111111','text': '#7FDBFF'}
tabs_styles = {'height': '30px'}
tab_style = {'borderBottom': '1px solid #d6d6d6',
             'padding': '6px'}
tab_selected_style = {'borderTop': '1px solid #d6d6d6',
                      'borderBottom': '1px solid #d6d6d6',
                      'backgroundColor': '#119DFF',
                      'fontWeight': 'bold',
                      'color': 'blue','padding': '6px'}


app = JupyterDash(__name__)
app.layout = html.Div([
    html.H4('Daily cases and deaths by date reported to WHO',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
    html.I("Choose a DatePickerRange to observe daily cases and deaths by date.",
           style={'textAlign': 'left', 'color': 'white', 'font-size': 15}),
    html.Hr(),
    dcc.DatePickerRange(id='my-date-picker-range',
                        minimum_nights=5,
                        min_date_allowed=data['Date_reported'].min(),
                        max_date_allowed=data['Date_reported'].max(),
                        display_format='Y-M-D',
                        start_date=data['Date_reported'].max().strftime("%Y-%m-%d"),
                        end_date=data['Date_reported'].max().strftime("%Y-%m-%d")),
    html.Hr(),
    html.Div(id='output-container-date-picker-range',style={'color': 'white', 'font-size': 12}),
    html.Br(),
    dcc.Tabs(id='tabs-1', value='New_cases', children=[
        dcc.Tab(label='New_cases', value='New_cases', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='New_deaths', value='New_deaths', style=tab_style, selected_style=tab_selected_style)]),
    html.Br(),
    dcc.Slider(20, 237, value=20, id='my-slider',marks={
        20: {'label': 'Top20', 'style': {'color': '#77b0b1'}},
        50: {'label': 'Top50', 'style': {'color': '#77b0b1'}},
        100: {'label': 'Top100', 'style': {'color': '#77b0b1'}},
        237: {'label': 'All', 'style': {'color': '#f50'}}},included=False),
        html.Div(id='slider-output-container',style={'color': 'Gold', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    dcc.Graph(id="graph"),
    ])

@app.callback(
    Output("output-container-date-picker-range", "children"),
    Input("my-date-picker-range", "start_date"),
    Input("my-date-picker-range", "end_date"),)
def number_render(start_date, end_date):
    return "From: {} To: {}.".format(start_date, end_date)

@app.callback(
    Output('slider-output-container', 'children'),
    Input('my-slider', 'value'))
def update_output(value):
  return 'You have selected top {} countries'.format(value)

@app.callback(Output("graph", "figure"),
              Input("my-date-picker-range", "start_date"),
              Input("my-date-picker-range", "end_date"),
              Input("tabs-1","value"),
              Input("my-slider", "value"))
def update_graph(start_date, end_date, tab, slider):
    df = data[(data['Date_reported']>=start_date) & (data['Date_reported']<=end_date)]
    bar_data = df.groupby('Country')[['New_cases','New_deaths']].sum().reset_index().sort_values(by = tab,ascending = False).head(slider)
    if slider <= 50:
        fig = px.bar(bar_data, x="Country", y=tab) 
        fig.update_layout(title_text='Total number of {} from {} to {} to the top countries split'.format(tab,start_date,end_date),
                          titlefont = dict(family = 'Rockwell',size = 15,color = 'Blue'),
                          font=dict(family='Rockwell', size=10, color='crimson'),
                          width = 2500,height=500)
        fig.update_xaxes(tickangle=45, title_text="Country",color='black')
        fig.update_yaxes(title_text='{}'.format(tab), titlefont = dict(family = 'Rockwell',size = 10,color = 'crimson'),)
        fig.add_annotation(x=0.01, y=round(bar_data[tab].max(),2)+(round(bar_data[tab].max(),2)*0.03),
                           text="{}-max: {}, {}-min: {}".format(tab,bar_data[tab].max(),tab,bar_data[tab].min()), 
                           showarrow=False,font=dict(color="black",size=8),)
        return fig
    elif slider <= 100:
        fig = px.bar(bar_data, x="Country", y=tab) 
        fig.update_layout(title_text='Total number of {} from {} to {} to the top countries split'.format(tab,start_date,end_date),
                          titlefont = dict(family = 'Rockwell',size = 15,color = 'Blue'),
                          font=dict(family='Rockwell', size=8, color='crimson'),
                          width = 2500,height=500)
        fig.update_xaxes(tickangle=45, title_text="Country",color='black')
        fig.update_yaxes(title_text='{}'.format(tab), titlefont = dict(family = 'Rockwell',size = 10,color = 'crimson'),)
        fig.add_annotation(x=0.01, y=round(bar_data[tab].max(),2)+(round(bar_data[tab].max(),2)*0.03),
                           text="{}-max: {}, {}-min: {}".format(tab,bar_data[tab].max(),tab,bar_data[tab].min()), 
                           showarrow=False,font=dict(color="black",size=8),)
        return fig
    elif slider > 100:
        fig = px.bar(bar_data, x="Country", y=tab) 
        fig.update_layout(title_text='Total number of {} from {} to {} to the top countries split'.format(tab,start_date,end_date),
                          titlefont = dict(family = 'Rockwell',size = 15,color = 'Blue'),
                          font=dict(family='Rockwell', size=6, color='crimson'),
                          width = 2500,height=500)
        fig.update_xaxes(tickangle=45, title_text="Country",color='black')
        fig.update_yaxes(title_text='{}'.format(tab), titlefont = dict(family = 'Rockwell',size = 10,color = 'crimson'),)
        fig.add_annotation(x=0.01, y=round(bar_data[tab].max(),2)+(round(bar_data[tab].max(),2)*0.03),
                           text="{}-max: {}, {}-min: {}".format(tab,bar_data[tab].max(),tab,bar_data[tab].min()), 
                           showarrow=False,font=dict(color="black",size=8),)
        return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>